## **GPU acceleration in FAISS**

FAISS can leverage GPUs to perform computations much faster than on CPUs. FAISS provides a GPU module that allows moving indices and the associated computations to GPUs. This is particularly useful for:
- Large-scale datasets with millions of vectors.
- Complex indices like IVF or PQ, where GPU parallelism significantly speeds up processing.


In [1]:
import numpy as np
import faiss
print("FAISS version:", faiss.__version__)
print("Has GPU support:", faiss.get_num_gpus() > 0)

FAISS version: 1.7.2
Has GPU support: True


### Example : Move an index to GPU
Here, we demonstrate how to move an index from the CPU to the GPU and perform a search using the GPU-accelerated index. We will start with a simple IndexFlatL2 example and move it to the GPU.

##### Step 1: Generate random dataset

In [2]:
# Generate a dataset of random vectors
dimension = 128
num_vectors = 10000
data = np.random.random((num_vectors, dimension)).astype('float32')

##### Step 2: Create a CPU index

In [3]:
# Create a CPU index
cpu_index = faiss.IndexFlatL2(dimension)

We create an `IndexFlatL2` index on the CPU. This is a simple, brute-force index that computes the L2 (Euclidean) distance to search for the nearest neighbors.

##### Step 3: Add data to the CPU index

In [4]:
# Add data to the CPU index
cpu_index.add(data)

We add the dataset (`data`) to the CPU-based index. This index will store all the vectors in memory for fast comparisons during a search.

##### Step 4: Move the index to GPU
We should move the index to the GPU after creating the index and adding data to it on the CPU. Moving the index to the GPU is done once and after the index is populated with data.

In [5]:
# Move the index to the GPU
gpu_index = faiss.index_cpu_to_gpu(faiss.StandardGpuResources(), 0, cpu_index)
print("Index moved to GPU.")

Index moved to GPU.


`faiss.index_cpu_to_gpu` is the key function that moves the index from the CPU to the GPU. It requires:
- `faiss.StandardGpuResources()`: This specifies the GPU resources FAISS will use.
- `0`: This indicates we are using GPU 0 (in case you have multiple GPUs).
- `cpu_index`: The index we created on the CPU.

After this step, the `gpu_index` is a GPU-accelerated version of the CPU index.

##### Step 5: Search on the GPU

In [6]:
# Perform a search on the GPU index
query_vector = np.random.random((1, dimension)).astype('float32')
distances, indices = gpu_index.search(query_vector, 5)
print("Indices of nearest neighbors:", indices)

Indices of nearest neighbors: [[7862 1563 2353 3671 3181]]


We create a random query vector, similar to the dataset vectors. The `search` function is used to find the nearest neighbors of the query vector in the GPU-based index (`gpu_index`).